In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/train.csv')

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.target.value_counts()

In [ ]:
data.tail()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data.head(5)

In [ ]:
X = data.drop(['ID_code','target'],axis = 1)

In [ ]:
X.shape

In [ ]:
y = data['target']

In [ ]:
y.shape

In [ ]:
### Steps to improve the performance

In [ ]:
len(y[y==0].index)

In [ ]:
### Normalize the data first

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
### Undersample values with y == 0

In [ ]:
print(X.shape)

In [ ]:
print(X.shape[0])

In [ ]:
n_train = X.shape[0]
print(n_train)
no_samples = int(0.6 * n_train)

In [ ]:
no_samples

In [ ]:
drop_indices = np.random.choice(y[y == 0].index,no_samples,replace = False)

In [ ]:
len(drop_indices)

In [ ]:
print('Shape b/f Undersampling')
print('X: {}, y:{}'.format(X.shape,y.shape))

In [ ]:
X = X.drop(drop_indices,axis = 0)
y = y.drop(drop_indices,axis = 0)

In [ ]:
print('Shape after undersampling')
print('X: {},y:{}'.format(X.shape,y.shape))

In [ ]:
len_y_0 = len(y[y==0].index)

In [ ]:
len_y_0

In [ ]:
len_y_1 = len(y[y==1].index)

In [ ]:
len_y_1

In [ ]:
y = pd.Series(y,name = 'target')

In [ ]:
y.value_counts()

In [ ]:
data = pd.concat([X,y],axis = 1).reset_index(drop = True)

In [ ]:
data.target.value_counts()

In [ ]:
dataset = data.sample(frac = 1).reset_index(drop = True)

In [ ]:
X = dataset.drop('target',axis = 1)

In [ ]:
y = dataset['target']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
from tensorflow.python.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.contrib.layers import l2_regularizer
from tensorflow.python.keras.optimizers import SGD

In [ ]:
model = Sequential()
#model.add(BatchNormalization(input_shape = (201)))
model.add(Dense(units = 2048,kernel_initializer='truncated_normal',activation='relu'))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())
model.add(Dense(units = 1024,activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(BatchNormalization())
model.add(Dense(units = 128,activation = 'relu'))
model.add(Dense(units = 8,activation = 'relu'))
model.add(Dense(units = 1,activation = 'sigmoid'))

In [ ]:
optimizer = SGD(lr = 0.01)

In [ ]:
callback = EarlyStopping(patience = 2)

In [ ]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
X_train_nn = np.array(X_train)
y_train_nn = np.array(y_train)

In [ ]:
model.fit(X_train_nn,y_train_nn,epochs = 10,validation_split = 0.05,batch_size = 32,callbacks = [callback],shuffle = True)

In [ ]:
data_sub = pd.read_csv('../input/test.csv')

In [ ]:
ids = data_sub['ID_code']

In [ ]:
features = data_sub.drop('ID_code',axis = 1)

In [ ]:
predictions = model.predict(features)

In [ ]:
pred_nn = np.array(predictions)
for i in range(len(pred_nn)):
    if pred_nn[i] >= 0.5:
        pred_nn[i] = 1
    else:
        pred_nn[i] = 0
pred_nn = np.array(pred_nn,dtype = np.int32)

In [ ]:
sub = pd.DataFrame()
sub['ID_code'] = ids
sub['target'] = pred_nn
sub.head()
sub.set_index('ID_code')
sub['target'].value_counts()
sub.to_csv('submissions.csv',index = False)